In [1]:
from ampligraph.datasets import load_wn18

In [2]:
X = load_wn18()

In [3]:
import numpy as np

In [4]:
filter = np.concatenate((X['train'], X['valid'], X['test']))

In [7]:
X['train'][0:3]

array([['03964744', '_hyponym', '04371774'],
       ['00260881', '_hypernym', '00260622'],
       ['02199712', '_member_holonym', '02188065']], dtype=object)

In [8]:
filter

array([['03964744', '_hyponym', '04371774'],
       ['00260881', '_hypernym', '00260622'],
       ['02199712', '_member_holonym', '02188065'],
       ...,
       ['15137890', '_hyponym', '15138401'],
       ['01762525', '_hyponym', '02216066'],
       ['01838038', '_hypernym', '01503061']], dtype=object)

In [173]:
import os
import pandas as pd
import re
import numpy as np
import glob
from nltk.corpus import stopwords
from xlrd import XLRDError
# nltk_stop_words = stopwords.words('english')
from stop_words import get_stop_words
from num2words import num2words 

from difflib import get_close_matches
from re import sub
# path='/Users/simranpal20gmail.com/Documents/Apple/Data/Week13/Manual_Match'

In [133]:
import os
# from xlrd import XLRDError
path="/Users/simranpal20gmail.com/Box/Data Operations Reporting/Consumer Reporting/SFO Exceptions Dashboard/SFO Exceptions Data Sources/SFO_Docs"
os.chdir(path)
store_dq=[f for f in os.listdir(path) if not f.startswith('.')]
store_dq
LastWeek_df = pd.read_excel(store_dq[3],sheet_name='Branded Data')
CurrentWeek_df=pd.read_excel(store_dq[5],sheet_name='Branded Fixtures')
# store_dq[1]

['Last_Week_BrandedFixtures_Test.xlsx',
 'Sample - Superstore.xls',
 'Sample - Superstore_1.xlsx',
 'SFO Exception Report - Tableau Data - 09.03.2021.xlsx',
 'SFO Exception Report Source Data.xlsm',
 'SFO Exception Reports - Data File v1.0.xlsx',
 'SFO_R_Script.R',
 'StoreFront Operations Exceptions.twb']

In [16]:
Store_df = pd.read_excel(store_dq[0])

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2018-152156,2018-11-08,2018-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2018-152156,2018-11-08,2018-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2018-138688,2018-06-12,2018-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2017-108966,2017-10-11,2017-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2017-108966,2017-10-11,2017-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2016-110422,2016-01-21,2016-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,33180.0,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028
9990,9991,CA-2019-121258,2019-02-26,2019-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627.0,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332
9991,9992,CA-2019-121258,2019-02-26,2019-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627.0,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932
9992,9993,CA-2019-121258,2019-02-26,2019-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627.0,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200


In [26]:
Prod_Names = Store_df['Product Name']
MarketShare = Store_df['Sales']

Prod_List = []
for i, j in zip(MarketShare, Prod_Names):
    if (i > 300) or (j == "HP Standard 104 key PS/2 Keyboard"):
        Prod_List.append(j)
    else:
        Prod_List.append("Other")

print(Prod_List)
print(len(Prod_List))
print(len(Prod_Names))


['Other', 'Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back', 'Other', 'Bretford CR4500 Series Slim Rectangular Table', 'Other', 'Other', 'Other', 'Mitel 5320 IP Phone VoIP phone', 'Other', 'Other', 'Chromcraft Rectangular Conference Tables', 'Konftel 250 Conference\xa0phone\xa0- Charcoal black', 'Other', 'Fellowes PB200 Plastic Comb Binding Machine', 'Other', 'Other', 'Stur-D-Stor Shelving, Vertical 5-Shelf: 72"H x 36"W x 18 1/2"D', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Bretford CR4500 Series Slim Rectangular Table', 'Other', 'Other', 'Riverside Palais Royal Lawyers Bookcase, Royale Cherry Finish', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'GE 30524EE4', 'Other', 'Other', 'Atlantic Metals Mobile 3-Shelf Bookcases, Custom Colors', 'Other', 'Plantronics HL10 Handset Lifter', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'AT&T CL83451 4-Handset Telephone', 'Oth

['Last_Week_BrandedFixtures_Test.xlsx',
 'Sample - Superstore.xls',
 'Sample - Superstore_1.xlsx',
 'SFO Exception Report - Tableau Data - 09.03.2021.xlsx',
 'SFO Exception Report Source Data.xlsm',
 'SFO Exception Reports - Data File v1.0.xlsx',
 'SFO_R_Script.R',
 'StoreFront Operations Exceptions.twb',
 '~$SFO Exception Reports - Data File v1.0.xlsx']

In [128]:
LastWeek_df = pd.read_excel(store_dq[3],sheet_name='Branded Data')
CurrentWeek_df=pd.read_excel(store_dq[5],sheet_name='Branded Fixtures')

In [124]:
CurrentWeek_df['Weeks Outstanding']=1
print(CurrentWeek_df[['Apple ID','Weeks Outstanding']])
print(CurrentWeek_df.shape)

       Apple ID  Weeks Outstanding
0     1003232.0                  1
1     1003233.0                  1
2     1003234.0                  1
3     1003235.0                  1
4     1003237.0                  1
...         ...                ...
6185   999387.0                  1
6186   999393.0                  1
6187   999405.0                  1
6188        NaN                  1
6189        NaN                  1

[6190 rows x 2 columns]
(6190, 25)


In [131]:
print(CurrentWeek_df[['Apple ID','Weeks Outstanding']])
print(CurrentWeek_df.shape)

      Apple ID  Weeks Outstanding
0      1003232                  1
1      1003233                  1
2      1003234                  1
3      1003235                  1
4      1003237                  1
...        ...                ...
6183    999385                  1
6184    999386                  1
6185    999387                  1
6186    999393                  1
6187    999405                  1

[6188 rows x 2 columns]
(6188, 25)


In [132]:
CurrentWeek_df['Weeks Outstanding'].value_counts()

1    6125
3      37
4      18
2       7
5       1
Name: Weeks Outstanding, dtype: int64

In [107]:
# LastWeek_df['Apple ID']=LastWeek_df['Apple ID'].astype('Int64')
# Current_Week_Apple_ID_ls=CurrentWeek_df['Apple ID'].dropna().value_counts().index.astype('Int64').tolist()
# for LW_index,LW_row in LastWeek_df.iterrows():
#     if LW_row['Apple ID'] in Current_Week_Apple_ID_ls:
#         LastWeek_df.at[LW_index,'Weeks Outstanding']=LW_row['Weeks Outstanding']+1
#     else:
#         LastWeek_df.at[LW_index,'Weeks Outstanding']=LW_row['Weeks Outstanding']

In [ ]:
# {k : orgs_dict[k] for k in orgs_dict.keys() if c[orgs_dict[k]]>1}
# Weeks_Outstanding=[v for k,v in LastWeek_Dict.items() if k == CW_row['Apple ID']]

In [129]:
LastWeek_df['Apple ID']=LastWeek_df['Apple ID'].dropna().astype('Int64')
LastWeek_df = LastWeek_df[LastWeek_df['Apple ID'].notna()]
CurrentWeek_df['Apple ID']=CurrentWeek_df['Apple ID'].dropna().astype('Int64')
CurrentWeek_df = CurrentWeek_df[CurrentWeek_df['Apple ID'].notna()]
CurrentWeek_df['Weeks Outstanding']=1
LastWeek_Dict = dict(zip(LastWeek_df['Apple ID'], LastWeek_df['Weeks Outstanding']))
Current_Week_Apple_ID_ls=CurrentWeek_df['Apple ID'].dropna().value_counts().index.astype('Int64').tolist()
for CW_index,CW_row in CurrentWeek_df.iterrows():
    Weeks_Outstanding=[v for k,v in LastWeek_Dict.items() if k == CW_row['Apple ID']]
    if Weeks_Outstanding:
        Weeks_Outstanding1="".join(map(str,Weeks_Outstanding))
        Weeks_Outstanding1=int(Weeks_Outstanding1)
        CurrentWeek_df.at[CW_index,'Weeks Outstanding']=Weeks_Outstanding1+1

In [139]:
# CurrentWeek_df['Apple ID']=CurrentWeek_df['Apple ID'].dropna().astype('Int64')
# CurrentWeek_df = CurrentWeek_df[CurrentWeek_df['Apple ID'].notna()]


 # 6188

In [ ]:
mylist = ['nowplaying', 'PBS', 'PBS', 'nowplaying', 'job', 'debate', 'thenandnow']
myset = set(mylist)
print(myset)
mynewlist = list(myset)

In [149]:

# cleanedList = [x for x in CurrentWeek_AppleId_ls if x != 'nan']

[1003232, 1003233, 1003234, 1003235, 1003237, 1003247, 1003251, 1010271, 1010271, 1010271, 1010342, 1011205, 1011232, 1011329, 1011350, 1012078, 1012079, 1012080, 1012081, 1017546, 1022845, 1023590, 1024081, 1025582, 1025582, 1029141, 1030386, 1031940, 1032460, 1037050, 1040584, 1040585, 1040586, 1040602, 1040611, 1041705, 1041762, 1041762, 1041764, 1041764, 1041782, 1041782, 1043559, 1047074, 1047074, 1047081, 1047081, 1048022, 1048022, 1052056, 105370, 105397, 105399, 105425, 105494, 105516, 105667, 1056843, 1056844, 1062340, 1063053, 1063053, 1063914, 1063914, 1065534, 1065554, 1065627, 1065673, 1067314, 1067314, 106785, 106785, 106800, 106804, 106804, 106810, 106815, 106819, 106821, 106821, 106826, 106827, 106832, 106834, 106834, 106845, 106858, 106859, 106861, 106864, 106867, 106869, 106871, 106871, 106872, 106877, 106878, 106885, 106888, 106900, 106900, 106902, 106902, 106905, 106907, 106918, 106921, 106921, 106922, 106922, 106922, 106922, 106923, 106923, 106924, 106926, 106926, 

In [157]:
CurrentWeek_AppleId_ls=CurrentWeek_df['Apple ID'].tolist()
print(type(CurrentWeek_AppleId_ls[8]))
CurrentWeek_AppleId_ls = list(filter(None, CurrentWeek_AppleId_ls))
cleanedList = [x for x in countries if x != 'nan']

<class 'numpy.int64'>


TypeError: 'numpy.ndarray' object is not callable

In [169]:
import math
CurrentWeek_AppleId_ls=[value for value in CurrentWeek_AppleId_ls if not math.isnan(value)]

In [167]:
CurrentWeek_AppleId_ls=CurrentWeek_df['Apple ID'].tolist()
for ind,val in enumerate(CurrentWeek_AppleId_int_ls):
    print(type(val))
#     if type(val)!='numpy.int64':
#         print("None type")
    
# CurrentWeek_AppleId_int_ls = [int(x) for x in CurrentWeek_AppleId_ls]

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class

In [158]:
import math
CurrentWeek_AppleId_ls=CurrentWeek_df['Apple ID'].tolist()
CurrentWeek_AppleId_ls=[value for value in CurrentWeek_AppleId_ls if not math.isnan(value)]
CurrentWeek_AppleId_int_ls = [int(x) for x in CurrentWeek_AppleId_ls]
CurrentWeek_WeeksOutstanding_ls=[1]*len(CurrentWeek_AppleId_ls)

LastWeek_df['Apple ID']=LastWeek_df['Apple ID'].dropna().astype('Int64')
LastWeek_df = LastWeek_df[LastWeek_df['Apple ID'].notna()]
LastWeek_Dict = dict(zip(LastWeek_df['Apple ID'], LastWeek_df['Weeks Outstanding']))

for CW_index,CW_row in enumerate(CurrentWeek_AppleId_ls):
    Weeks_Outstanding=[v for k,v in LastWeek_Dict.items() if k == CW_row]
    if Weeks_Outstanding:
        Weeks_Outstanding1="".join(map(str,Weeks_Outstanding))
        Weeks_Outstanding1=int(Weeks_Outstanding1)
        CurrentWeek_WeeksOutstanding_ls[CW_index]=Weeks_Outstanding1+1

print(CurrentWeek_WeeksOutstanding_ls)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [180]:
import pandas as pd
from openpyxl import load_workbook

def write_excel(filename,sheetname,dataframe):
    with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer: 
        workBook = writer.book
        try:
            workBook.remove(workBook[sheetname])
        except:
            print("Worksheet does not exist")
        finally:
            dataframe.to_excel(writer, sheet_name=sheetname,index=False)
            writer.save()

In [179]:
import os
# from xlrd import XLRDError
path="/Users/simranpal20gmail.com/Box/Data Operations Reporting/Consumer Reporting/SFO Exceptions Dashboard/SFO Exceptions Data Sources/Test_Path"
os.chdir(path)
store_dq=[f for f in os.listdir(path) if not f.startswith('.')]
store_dq
LastWeek_df = pd.read_excel(store_dq[0],sheet_name='Branded Data')
CurrentWeek_df=pd.read_excel(store_dq[1],sheet_name='Branded Fixtures')
# store_dq[1]

In [181]:
CurrentWeek_AppleId_ls=CurrentWeek_df['Apple ID'].dropna().astype('Int64').tolist()
CurrentWeek_WeeksOutstanding_ls=[1]*len(CurrentWeek_AppleId_ls)

LastWeek_df['Apple ID']=LastWeek_df['Apple ID'].dropna().astype('Int64')
LastWeek_df = LastWeek_df[LastWeek_df['Apple ID'].notna()]
LastWeek_Dict = dict(zip(LastWeek_df['Apple ID'], LastWeek_df['Weeks Outstanding']))

for CW_index,CW_row in enumerate(CurrentWeek_AppleId_ls):
    Weeks_Outstanding=[v for k,v in LastWeek_Dict.items() if k == CW_row]
    if Weeks_Outstanding:
        Weeks_Outstanding1="".join(map(str,Weeks_Outstanding))
        Weeks_Outstanding1=int(Weeks_Outstanding1)
        CurrentWeek_WeeksOutstanding_ls[CW_index]=Weeks_Outstanding1+1
Final_df = pd.DataFrame(list(zip(CurrentWeek_AppleId_ls, CurrentWeek_WeeksOutstanding_ls)), columns = ['Apple ID', 'Weeks Outstanding'])
write_excel('SFO Exception Report - Tableau Data.xlsx','Branded Data',Final_df)

1

In [57]:
NewWeek_df=LastWeek_df.merge(CurrentWeek_df, how='left',on='Apple ID')
NewWeek_df

,Apple ID,Store ID_x,Account Status CD_x,Country/Region Name_x,Region Name_x,Sub Geo Name_x,Reseller Name_x,Store Name_x,Store City Name_x,Store Program ID_x,...,Project Type Desc_y,Fixture Type Desc_y,Fixture Status Group_y,Fixture Status Description_y,Target Installation Date_y,Target Opening Date_y,Target Closure Date_y,Actual Installation Date_y,Actual Opening Date(Fixture)_y,Actual Closure Date(Fixture)_y
0,1029149.0,5961879,ACTIVE,TURKEY,EASTERN EUROPE,IMMEA,TURKCELL İLETIŞIM HIZMETLERI A.Ş.,ŞENGÜLLER MAĞAZACILIK A.Ş,İSTANBUL,108516872,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
1,1516262.0,60356261,ACTIVE,GERMANY,DACH,WESTERN EUROPE,TELEKOM DEUTSCHLAND GMBH,"TELEKOM SHOP BAD KREUZNACH, MANNHEIMER STRAßE 163",BAD KREUZNACH,108643854,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
2,1748843.0,60351752,ACTIVE,UNITED ARAB EMIRATES,MEA,IMMEA,ETISALAT,ETISALAT - DEIRA CC-2 D92,DUBAI,108540312,...,New,Free-Standing,Open,17.0 Open & Operational,2016-01-14,2016-01-14,NaT,2016-01-14,2016-01-14,NaN
3,2291464.0,60475861,ACTIVE,RUSSIA,EASTERN EUROPE,IMMEA,M.VIDEO,M.VIDEO,MYTISHCHI,108674319,...,New,Inline Back to Back,Pre-Opening,13.0 Install Date Pending,2021-03-26,2021-03-26,NaT,NaT,NaT,NaN
4,422615.0,561990,ACTIVE,PORTUGAL,SOUTH EUROPE,WESTERN EUROPE,"WORTEN, EQUIPAMENTOS PARA O LAR, SA",WORTEN OEIRAS MGS LC,OEIRAS,108627112,...,New,2 Free-Standing Tables,Pre-Opening,7.0 Design Kit Submission,2021-05-14,2021-05-14,NaT,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,3377269.0,60548517,ACTIVE,SAUDI ARABIA,MEA,IMMEA,STC CHANNELS,STC - AL MUNTAZAH,AL KHOBAR,108679574,...,New,Wall Bay Shop,Pre-Opening,7.0 Design Kit Submission,2021-08-13,2021-08-13,NaT,NaT,NaT,NaN
85,520315.0,5821890,ACTIVE,FRANCE,SOUTH EUROPE,WESTERN EUROPE,FNAC DARTY PARTICIPATIONS ET SERVICES SIÈGE SO...,FNAC REIMS THILLOIS,THILLOIS,10807819,...,New,Inline Back to Back,Open,15.0 Open & Operational: Post Install Submission,2016-03-17,2016-03-17,NaT,2016-02-11,2016-02-11,NaN
86,520315.0,5821890,ACTIVE,FRANCE,SOUTH EUROPE,WESTERN EUROPE,FNAC DARTY PARTICIPATIONS ET SERVICES SIÈGE SO...,FNAC REIMS THILLOIS,THILLOIS,10807819,...,New,Feature Inline (cross program),Open,17.0 Open & Operational,2018-06-18,2018-06-18,NaT,2018-06-18,2018-06-18,NaN
87,520315.0,5821890,ACTIVE,FRANCE,SOUTH EUROPE,WESTERN EUROPE,FNAC DARTY PARTICIPATIONS ET SERVICES SIÈGE SO...,FNAC REIMS THILLOIS,THILLOIS,10807819,...,New,Feature Wall Bay,Pre-Closing,18.0 Scheduled for Relocation,2019-09-25,2019-09-25,2021-03-08,2019-09-25,2019-09-25,NaN
